# Covariance matrix emulator

The covariance matrix emulator predicts a covariance matrix given a set of input covariance matrices that depend on some underlying variables. This notebook tests the emulator on a simple model - linear scaling of the covariance matrix on a constant.
$$
C(A) = AC_0\,,
$$
where $A$ is the scale parameter, and $C_0$ is some fiducial covariance matrix.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import covariance_emulator as ce
%matplotlib inline

## Test 1 - scaling the identity matrix

The most simple example will have $C_0 = \mathbf{I}$.

In [2]:
ndim = 6 #Covariances will be ndim X ndim in size
parameters = np.arange(1,10)
Cs = np.array([p*np.identity(ndim) for p in parameters])

In [3]:
Emu = ce.CovEmu(parameters, Cs)

((9, 6), (6,), (6, 6), (9, 6))
((9, 9), (9,), (9, 15), (9, 15))


In [4]:
test_parameter = 5.5
Cpredicted = Emu.predict(test_parameter)
print(Cpredicted)

[[5.49779516 0.         0.         0.         0.         0.        ]
 [0.         5.49779516 0.         0.         0.         0.        ]
 [0.         0.         5.49779516 0.         0.         0.        ]
 [0.         0.         0.         5.49779516 0.         0.        ]
 [0.         0.         0.         0.         5.49779516 0.        ]
 [0.         0.         0.         0.         0.         5.49779516]]


In [5]:
Ctrue = test_parameter * np.identity(ndim)
diff = Ctrue - Cpredicted
print(diff[np.nonzero(diff)]/Ctrue.diagonal())

[0.00040088 0.00040088 0.00040088 0.00040088 0.00040088 0.00040088]


## Success!

The elements of the covariance are predicted to  ~0.04%.

## Test 2 - scaling a matrix with linearly increasing variances

Let's suppose that the diagonal is now a linear function, e.g.
$$
C_{0}^{ii} = i+1
$$
and we have the same scaling with $A$. Let's try to recover the true matrix.

In [6]:
Cs = np.array([p*np.diag(np.arange(1,ndim+1)) for p in parameters])

In [7]:
Emu = ce.CovEmu(parameters, Cs)
test_parameter = 5.5
Cpredicted = Emu.predict(test_parameter)
Ctrue = test_parameter * np.diag(np.arange(1,ndim+1))
diff = Ctrue - Cpredicted
print(diff[np.nonzero(diff)]/Ctrue.diagonal())

((9, 6), (6,), (6, 6), (9, 6))
((9, 9), (9,), (9, 15), (9, 15))
[0.00040088 0.00040088 0.00040088 0.00040088 0.00040088 0.00040088]


In [8]:
Emu = ce.CovEmu(parameters, Cs, NPC_D=2)
test_parameter = 5.5
Cpredicted = Emu.predict(test_parameter)
Ctrue = test_parameter * np.diag(np.arange(1,ndim+1))
diff = Ctrue - Cpredicted
print(diff[np.nonzero(diff)]/Ctrue.diagonal())

((9, 6), (6,), (6, 6), (9, 6))
((9, 9), (9,), (9, 15), (9, 15))
[0.00040088 0.00040088 0.00040088 0.00040088 0.00040088 0.00040088]


## Interesting - required more Principle Components

Notice that the emulator did awful when we built it with the default number of principle components (one), but much better when we used two principle components. This makes sense, since as the covariance matrix (specifically, the diagonal) had more structure it requires more PCs to describe its behavior.

## Test 3 - scaling a matrix with exponentially increasing variances

Real astronomical data is measured over a range of scales. Because of this, errorbars can be orders of magnitude different. Let's pretend that the variance has the form
$$
C_{0}^{ii} = e^i
$$
and we have the same scaling with $A$. Let's try to recover the true matrix again.

In [9]:
Cs = np.array([p*np.diag(np.exp(np.arange(1,ndim+1))) for p in parameters])

In [10]:
Emu = ce.CovEmu(parameters, Cs)
test_parameter = 5.5
Cpredicted = Emu.predict(test_parameter)
Ctrue = test_parameter * np.diag(np.exp(np.arange(1,ndim+1)))
diff = Ctrue - Cpredicted
print(diff[np.nonzero(diff)]/Ctrue.diagonal())

((9, 6), (6,), (6, 6), (9, 6))
((9, 9), (9,), (9, 15), (9, 15))
[0.00040088 0.00040088 0.00040088 0.00040088 0.00040088 0.00040088]


### Bad

We need more principle components. But even still, the emulator predicts at the ~25% level.

In [11]:
Emu = ce.CovEmu(parameters, Cs, NPC_D=2)
test_parameter = 5.5
Cpredicted = Emu.predict(test_parameter)
Ctrue = test_parameter * np.diag(np.exp(np.arange(1,ndim+1)))
diff = Ctrue - Cpredicted
print(diff[np.nonzero(diff)]/Ctrue.diagonal())

((9, 6), (6,), (6, 6), (9, 6))
((9, 9), (9,), (9, 15), (9, 15))
[0.00040088 0.00040088 0.00040088 0.00040088 0.00040088 0.00040088]


### Success again

We can see that despite different behaviors of the diagonals of the covariance matrices, the emulator can successfully make predictions, as long as there are enough principle components.